In [ ]:
import gc
import os

import numpy as np
import open3d as o3d

from lead.common import common_utils
from lead.common.constants import (
    SEMANTIC_SEGMENTATION_CONVERTER,
    TRANSFUSER_SEMANTIC_COLORS,
    CameraPointCloudIndex,
    CarlaSemanticSegmentationClass,
    TransfuserSemanticSegmentationClass,
)

for frame_id in range(200, 500):
    root = "data/expert_debug/data/debug_routes/Town15_Rep-1_1_town15_construction_route0_01_15_12_48_52"

    pc_path = f"{root}/camera_pc/{str(frame_id).zfill(4)}.npz"
    bb_path = f"{root}/bboxes/{str(frame_id).zfill(4)}.pkl"

    points_ego = np.load(pc_path)["arr_0"]
    boxes = common_utils.read_pickle(bb_path)

    # 3D visualization with Open3D
    pcd = o3d.geometry.PointCloud()

    # Ensure points_ego is in the correct shape (N, 3) and dtype
    if points_ego.shape[0] == 3 or points_ego.shape[0] == 4:
        # If shape is (3, N) or (4, N), transpose to (N, 3) or (N, 4)
        points_ego = points_ego.T

    # Extract xyz coordinates and semantic class using the proper indices
    points_xyz = points_ego[:, [CameraPointCloudIndex.X, CameraPointCloudIndex.Y, CameraPointCloudIndex.Z]].astype(np.float64)
    carla_semantic_classes = points_ego[:, CameraPointCloudIndex.UNREAL_SEMANTICS_ID].astype(np.int32)

    # Flip Y-axis to convert from CARLA to Open3D coordinate system
    points_xyz[:, 1] = -points_xyz[:, 1]

    # Convert CARLA semantic classes to TransFuser semantic classes
    transfuser_semantic_classes = np.array(
        [
            SEMANTIC_SEGMENTATION_CONVERTER.get(
                CarlaSemanticSegmentationClass(c), TransfuserSemanticSegmentationClass.UNLABELED
            )
            for c in carla_semantic_classes
        ],
        dtype=np.int32,
    )

    # Filter out points that are too far away (> 100m)
    distances = np.linalg.norm(points_xyz, axis=1)
    mask = distances <= 100.0
    points_xyz = points_xyz[mask]
    transfuser_semantic_classes = transfuser_semantic_classes[mask]

    # Convert TransFuser colors from RGB (0-255) to normalized [0-1] for Open3D
    TRANSFUSER_SEMANTIC_COLORS[TransfuserSemanticSegmentationClass.UNLABELED] = [60, 60, 60]  # Grey unlabeled
    transfuser_colors_normalized = {
        k: [v[0] / 255.0, v[1] / 255.0, v[2] / 255.0] for k, v in TRANSFUSER_SEMANTIC_COLORS.items()
    }

    # Assign colors to points based on TransFuser semantic class
    colors = np.array(
        [
            transfuser_colors_normalized.get(
                TransfuserSemanticSegmentationClass(int(c)),
                [0.0, 0.0, 0.0],  # Black for unlabeled
            )
            for c in transfuser_semantic_classes
        ]
    )

    pcd.points = o3d.utility.Vector3dVector(points_xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Create line set for bounding boxes - SINGLE LOOP
    all_points = []
    lines_list = []
    colors_list = []

    for box in boxes:
        # Get offset BEFORE adding points
        offset = len(all_points)

        # Create box corners
        extent = np.array(box["extent"])
        center = np.array(box["position"])
        yaw = box["yaw"]

        # Flip Y-axis for bounding box center
        center[1] = -center[1]
        # Flip yaw angle for Y-axis flip
        yaw = -yaw

        # Define the 8 corners of the box in local coordinates
        corners_local = np.array(
            [
                [-extent[0], -extent[1], -extent[2]],
                [extent[0], -extent[1], -extent[2]],
                [extent[0], extent[1], -extent[2]],
                [-extent[0], extent[1], -extent[2]],
                [-extent[0], -extent[1], extent[2]],
                [extent[0], -extent[1], extent[2]],
                [extent[0], extent[1], extent[2]],
                [-extent[0], extent[1], extent[2]],
            ]
        )

        # Rotation matrix
        R = np.array([[np.cos(yaw), -np.sin(yaw), 0], [np.sin(yaw), np.cos(yaw), 0], [0, 0, 1]])

        # Transform corners to world coordinates
        corners_world = (R @ corners_local.T).T + center

        # Add corners NOW
        all_points.extend(corners_world)

        # Define the 12 edges of the box
        edges = [
            [0, 1],
            [1, 2],
            [2, 3],
            [3, 0],  # bottom face
            [4, 5],
            [5, 6],
            [6, 7],
            [7, 4],  # top face
            [0, 4],
            [1, 5],
            [2, 6],
            [3, 7],  # vertical edges
        ]

        # Color based on class
        if box.get("class") == "ego_car":
            color = [0, 1, 0]  # Green for ego
        elif box.get("transfuser_semantics_id") == TransfuserSemanticSegmentationClass.VEHICLE:
            color = [1.0, 0.5, 0]  # Red for vehicles
        elif box.get("transfuser_semantics_id") == TransfuserSemanticSegmentationClass.PEDESTRIAN:
            color = [0, 0, 1]  # Blue for pedestrians
        else:
            color = [1, 1, 0]  # Yellow for others

        # Add edges with offset
        for edge in edges:
            lines_list.append([offset + edge[0], offset + edge[1]])
            colors_list.append(color)

    # Create line set geometry
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(np.array(all_points))
    line_set.lines = o3d.utility.Vector2iVector(np.array(lines_list))
    line_set.colors = o3d.utility.Vector3dVector(np.array(colors_list))

    # Use OffscreenRenderer for proper offscreen rendering
    output_path = f"outputs/point_cloud_visualization/{str(frame_id).zfill(4)}.png"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    width, height = 640, 659
    renderer = o3d.visualization.rendering.OffscreenRenderer(width, height)

    # Setup material
    material = o3d.visualization.rendering.MaterialRecord()
    material.shader = "defaultUnlit"
    material.point_size = 1.5  # Make points smaller

    # Add point cloud
    renderer.scene.add_geometry("pcd", pcd, material)

    # Add bounding boxes
    material_lines = o3d.visualization.rendering.MaterialRecord()
    material_lines.shader = "unlitLine"
    material_lines.line_width = 1.5
    renderer.scene.add_geometry("boxes", line_set, material_lines)

    # Set camera view relative to ego
    camera_pos = np.array([-40, 30, 60])  # Behind and above ego
    look_at = np.array([0, 0, 0])  # Look at ego
    up_vector = np.array([0, 0, 1])  # Z is up

    renderer.setup_camera(60.0, look_at, camera_pos, up_vector)

    # Set background color
    renderer.scene.set_background([1.0, 1.0, 1.0, 1.0])  # White
    renderer.scene.view.set_post_processing(False)

    # Render and save
    img = renderer.render_to_image()
    o3d.io.write_image(output_path, img)
    del renderer
    del pcd
    del line_set
    gc.collect()

In [ ]:
from pathlib import Path
from PIL import Image

def create_webp_gif(image_dir, output_path, duration=500):
    """
    Create an animated WebP from images, using only the left half of each image.
    
    Args:
        image_dir: Directory containing the images
        output_path: Path to save the output WebP file
        duration: Duration of each frame in milliseconds
    """
    image_dir = Path(image_dir)
    image_files = sorted(image_dir.glob("*.png"))
    
    if not image_files:
        print(f"No images found in {image_dir}")
        return
    
    frames = []
    for img_path in image_files:
        img = Image.open(img_path)
        frames.append(img)
    
    if frames:
        # Save as animated WebP
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=duration,
            loop=0,
            compression=9,
            quality=30
        )
        print(f"Created animated WebP with {len(frames)} frames: {output_path}")
    
# Create the animated WebP
create_webp_gif("outputs/point_cloud_visualization", "outputs/point_cloud_visualization.webp", duration=50)